In [3]:
import os
import re
import plotly.graph_objects as go
import plotly.express as px
from typing import List, Tuple, Dict
import pandas as pd
import json
import networkx as nx

In [4]:


def parseDpdFile(filePath: str) -> Tuple[Dict[str, Dict[str, str]], List[Tuple[str, str]]]:
    nodes: Dict[str, Dict[str, str]] = {}
    edges: List[Tuple[str, str]] = []
    with open(filePath, 'r', encoding='utf-8') as file:
        for line in file:
            line = line.strip()
            if line.startswith("N:"):
                parts = line.split(maxsplit=3)
                nodeId: str = parts[1]
                label: str = parts[2].strip('"')
                attributes: str = parts[3] if len(parts) > 3 else ""
                nodes[nodeId] = {"label": label, "attributes": attributes}
            elif line.startswith("E:"):
                parts = line.split()
                source: str = parts[1]
                target: str = parts[2]
                edges.append((source, target))
    return nodes, edges

def createGraphFromDpd(filePath: str) -> nx.DiGraph:
    nodes, edges = parseDpdFile(filePath)
    G = nx.DiGraph()
    for nodeId, data in nodes.items():   
        G.add_node(nodeId, **data)
    G.add_edges_from(edges)
    return G

dpdFilePath: str = r"C:\Users\User\Documents\GitHub\autoformalization\src\dataset\dependency_graph\dgraph.dpd"
G = createGraphFromDpd(dpdFilePath)

with open(r"C:\Users\User\Documents\GitHub\autoformalization\src\dataset\processed_data\coq_proofs_dataset.json", "r") as file:
    json_data = json.load(file)

data = []
for entry in json_data:
    file_name = entry.get("fileName", "")
    items = entry.get("items", [])
    for item in items:
        raw_text = item.get("raw", "")
        item_type = item.get("type", "")
        data.append({"fileName": file_name, "type": item_type, "raw": raw_text})

df = pd.DataFrame(data)

def get_second_word(text: str) -> str:
    if not text:
        return ""
    tokens = text.split()
    if len(tokens) >= 2:
        return tokens[1].replace(':', '')
    return ""

df["Label"] = df["raw"].apply(get_second_word)

def parse_attributes(attr_str: str) -> dict:
    attr_str = attr_str.strip("[];")
    attr_pairs = attr_str.split(", ")
    attr_dict = {}
    for pair in attr_pairs:
        if "=" in pair:
            key, value = pair.split("=")
            attr_dict[key.strip()] = value.strip().strip('"')
    return attr_dict

expanded_data = []
for node, attributes in G.nodes(data=True):
    attrs = parse_attributes(attributes["attributes"])
    expanded_data.append({"Node": node, "Label": attributes["label"], **attrs})

df_dpd = pd.DataFrame(expanded_data)

def extract_filename(path: str) -> str:
    if not path:
        return ""
    return path.split(".")[0] + ".v"

df_dpd["file"] = df_dpd["path"].apply(extract_filename)

# Topological sort and add levels
level_dict = {}
for i, generation in enumerate(nx.topological_generations(G)):
    for n in generation:
        level_dict[n] = i

df_dpd["level"] = df_dpd["Node"].map(level_dict).fillna(pd.NA).astype("Int64")

merged_df = df.merge(
    df_dpd[["file", "Label", "Node", "level"]],
    left_on=["fileName", "Label"],
    right_on=["file", "Label"],
    how="left"
)

merged_df.drop(columns=["file"], inplace=True)
merged_df["Node"] = merged_df["Node"].fillna(pd.NA)
unmatched_df_dpd = df_dpd[
    ~df_dpd.set_index(["file", "Label"]).index.isin(df.set_index(["fileName", "Label"]).index)
]

merged_df

df = merged_df.dropna(subset=["Node"])
df = df.sort_values(by=["level", "fileName"])
df


,fileName,type,raw,Label,Node,level
4,AltAuto.v,Lemma,Lemma re_opt_e_match : forall T (re: reg_exp T...,re_opt_e_match,93,0
5,AltAuto.v,Theorem,"Theorem silly1 : forall n, 1 + n = S n.\nProof...",silly1,101,0
6,AltAuto.v,Theorem,"Theorem silly2 : forall (P : Prop), P -> P.\nP...",silly2,102,0
7,AltAuto.v,Lemma,"Lemma simple_semi : forall n, (n + 1 =? 0) = f...",simple_semi,103,0
9,AltAuto.v,Lemma,"Lemma simple_semi' : forall n, (n + 1 =? 0) = ...",simple_semi',104,0
...,...,...,...,...,...,...
852,ImpParser.v,Definition,Definition firstExpect {T} (t : token) (p : pa...,firstExpect,501,8
845,ImpParser.v,misc,Arguments SomeE {X}.\nArguments NoneE {X}.,SomeE,534,9
849,ImpParser.v,Definition,Definition parser (T : Type) :=\n list token ...,parser,525,9
840,ImpParser.v,Definition,Definition token := string.,token,528,10


In [5]:
pivot = pd.crosstab(df['fileName'], df['level'], margins=True, margins_name="Total")
pivot


level,0,1,2,3,4,5,6,7,8,9,10,Total
fileName,,,,,,,,,,,,
AltAuto.v,100,6,1,0,0,0,0,0,0,0,0,107
AltAutoTest.v,1,0,0,0,0,0,0,0,0,0,0,1
Auto.v,15,1,1,1,0,0,0,0,0,0,0,18
Basics.v,79,23,15,4,2,0,1,0,0,0,0,124
Imp.v,46,11,11,4,4,5,0,0,0,0,0,81
ImpCEvalFun.v,7,4,1,0,0,0,0,0,0,0,0,12
ImpParser.v,7,1,3,3,2,5,1,4,5,2,2,35
IndPrinciples.v,19,8,8,3,3,0,0,0,0,0,0,41
IndProp.v,110,24,22,5,2,0,0,0,0,0,0,163


In [6]:

def modifyDpdFileInPlace(filePath: str) -> None:
    with open(filePath, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    with open(filePath, 'w', encoding='utf-8') as file:
        for line in lines:
            match = re.search(r'path="([^"]+)"', line)
            if match:
                pathValue = match.group(1)
                fileName = pathValue.split('.')[0]
                if 'file=' not in line:
                    modifiedLine = line.rstrip()[:-2] + f', file="{fileName}", ];\n'
                    file.write(modifiedLine)
                else:
                    file.write(line)
            else:
                file.write(line)

def parseDpdFile(filePath: str) -> Tuple[Dict[str, Dict[str, str]], List[Tuple[str, str]]]:
    nodes: Dict[str, Dict[str, str]] = {}
    edges: List[Tuple[str, str]] = []

    with open(filePath, 'r', encoding='utf-8') as file:
        for line in file:
            line = line.strip()
            if line.startswith("N:"):
                parts = line.split(maxsplit=3)
                nodeId: str = parts[1]
                label: str = parts[2].strip('"')
                attributes: str = parts[3] if len(parts) > 3 else ""
                nodes[nodeId] = {"label": label, "attributes": attributes}
            elif line.startswith("E:"):
                parts = line.split()
                source: str = parts[1]
                target: str = parts[2]
                edges.append((source, target))

    return nodes, edges

def createGraphFromDpd(filePath: str) -> nx.DiGraph:
    nodes, edges = parseDpdFile(filePath)
    graph = nx.DiGraph()
    for nodeId, data in nodes.items():
        graph.add_node(nodeId, **data)
    graph.add_edges_from(edges)
    return graph

def plotGraphSpringLayout(graph: nx.DiGraph, title: str = "Dependency Graph (Spring Layout)") -> None:
    pos = nx.spring_layout(graph, seed=42, k=3 / (len(graph.nodes()) ** 0.5))

    edgeX, edgeY = [], []
    for src, dst in graph.edges():
        x0, y0 = pos[src]
        x1, y1 = pos[dst]
        edgeX.extend([x0, x1, None])
        edgeY.extend([y0, y1, None])

    edgeTrace = go.Scatter(x=edgeX, y=edgeY, line=dict(width=0.5, color="gray"), hoverinfo="none", mode="lines")

    nodeX, nodeY, hoverText = [], [], []
    for node in graph.nodes():
        x, y = pos[node]
        nodeX.append(x)
        nodeY.append(y)
        hoverText.append(f"ID: {node}<br>Label: {graph.nodes[node]['label']}<br>Attributes: {graph.nodes[node].get('attributes', 'None')}")

    nodeTrace = go.Scatter(x=nodeX, y=nodeY, mode="markers", hoverinfo="text", text=hoverText,
                           marker=dict(size=10, color="blue", line=dict(width=2)))

    fig = go.Figure(data=[edgeTrace, nodeTrace],
                    layout=go.Layout(title=title, titlefont_size=16, showlegend=False, hovermode="closest",
                                     width=1600, height=900, margin=dict(b=0, l=0, r=0, t=40),
                                     xaxis=dict(showgrid=False, zeroline=False),
                                     yaxis=dict(showgrid=False, zeroline=False)))
    fig.show()

def plotNodeDegreeHistogram(filePath: str) -> None:
    graph = createGraphFromDpd(filePath)
    degrees = [deg for _, deg in graph.degree()]

    fig = go.Figure(data=[go.Histogram(x=degrees, nbinsx=max(degrees) + 1, marker=dict(color='blue'))],
                    layout=go.Layout(title="Node Degree Histogram", xaxis_title="Degree", yaxis_title="Count"))
    fig.show()

def plotGraphUniqueFileColors(filePath: str) -> None:
    graph = createGraphFromDpd(filePath)
    pos = nx.spring_layout(graph, seed=42, k=3 / (len(graph.nodes()) ** 0.5))

    fileColors: Dict[str, str] = {}
    colorPalette = px.colors.qualitative.Plotly
    colorIndex: int = 0
    nodeColors: List[str] = []

    for node in graph.nodes():
        attr = graph.nodes[node].get("attributes", "")
        match = re.search(r'file="([^"]+)"', attr)
        fileName = match.group(1) if match else "Unknown"
        if fileName not in fileColors:
            fileColors[fileName] = colorPalette[colorIndex % len(colorPalette)]
            colorIndex += 1
        nodeColors.append(fileColors[fileName])

    edgeX, edgeY = [], []
    for src, dst in graph.edges():
        x0, y0 = pos[src]
        x1, y1 = pos[dst]
        edgeX.extend([x0, x1, None])
        edgeY.extend([y0, y1, None])

    edgeTrace = go.Scatter(x=edgeX, y=edgeY, line=dict(width=0.5, color="gray"), hoverinfo="none", mode="lines")

    nodeX, nodeY, hoverText = [], [], []
    for node in graph.nodes():
        x, y = pos[node]
        nodeX.append(x)
        nodeY.append(y)
        attr = graph.nodes[node].get("attributes", "None")
        hoverText.append(f"ID: {node}<br>Label: {graph.nodes[node]['label']}<br>Attributes: {attr}")

    nodeTrace = go.Scatter(x=nodeX, y=nodeY, mode="markers", hoverinfo="text", text=hoverText,
                           marker=dict(size=10, color=nodeColors, line=dict(width=2)))

    fig = go.Figure(data=[edgeTrace, nodeTrace],
                    layout=go.Layout(title="Dependency Graph (Colored by File)", titlefont_size=16, showlegend=False,
                                     hovermode="closest", width=1600, height=900, margin=dict(b=0, l=0, r=0, t=40),
                                     xaxis=dict(showgrid=False, zeroline=False),
                                     yaxis=dict(showgrid=False, zeroline=False)))
    fig.show()

def plotTopologicallySortedGraph(filePath: str) -> None:
    graph = createGraphFromDpd(filePath)

    # Ensure the graph is a DAG (Directed Acyclic Graph)
    if not nx.is_directed_acyclic_graph(graph):
        raise ValueError("The graph contains cycles and cannot be topologically sorted.")

    # Perform topological sort
    topo_order = list(nx.topological_sort(graph))

    # Assign levels to nodes
    level_map = {node: 0 for node in graph.nodes()}
    for node in topo_order:
        preds = list(graph.predecessors(node))
        if preds:
            level_map[node] = max(level_map[pred] for pred in preds) + 1

    # Create a horizontal layout based on topological order and level
    # Multiply x by 30 and y by 80 for spacing
    pos = {
        node: (i * 30, -level_map[node] * 80)
        for i, node in enumerate(topo_order)
    }

    # -------------------------------------------------------------------------
    # STEP 1: Center the layout
    # -------------------------------------------------------------------------
    xs = [p[0] for p in pos.values()]
    ys = [p[1] for p in pos.values()]
    if xs and ys:
        min_x, max_x = min(xs), max(xs)
        min_y, max_y = min(ys), max(ys)
        # Offset so that the midpoint is around (0, 0)
        offset_x = (max_x + min_x) / 2.0
        offset_y = (max_y + min_y) / 2.0

        for node in pos:
            x_old, y_old = pos[node]
            pos[node] = (x_old - offset_x, y_old - offset_y)

    # -------------------------------------------------------------------------
    # STEP 2: Use the same discrete color scheme by file name as plotGraphUniqueFileColors
    # -------------------------------------------------------------------------
    file_colors: Dict[str, str] = {}
    color_palette = px.colors.qualitative.Plotly
    color_index = 0

    # Collect edge coordinates
    edgeX, edgeY = [], []
    for src, dst in graph.edges():
        x0, y0 = pos[src]
        x1, y1 = pos[dst]
        edgeX.extend([x0, x1, None])
        edgeY.extend([y0, y1, None])

    edgeTrace = go.Scatter(
        x=edgeX,
        y=edgeY,
        line=dict(width=0.5, color="gray"),
        hoverinfo="none",
        mode="lines"
    )

    # Prepare node data
    nodeX, nodeY, hoverText = [], [], []
    nodeColors = []
    for node in topo_order:
        x, y = pos[node]
        nodeX.append(x)
        nodeY.append(y)

        # Extract file name from attributes
        attr = graph.nodes[node].get("attributes", "")
        match = re.search(r'file="([^"]+)"', attr)
        fileName = match.group(1) if match else "Unknown"

        # Assign color based on file name
        if fileName not in file_colors:
            file_colors[fileName] = color_palette[color_index % len(color_palette)]
            color_index += 1
        fill_color = file_colors[fileName]

        # Hover info
        hoverText.append(
            f"ID: {node}"
            f"<br>Label: {graph.nodes[node]['label']}"
            f"<br>File: {fileName}"
            f"<br>Level: {level_map[node]}"
            f"<br>Attributes: {attr if attr else 'None'}"
        )

        nodeColors.append(fill_color)

    # Node scatter
    nodeTrace = go.Scatter(
        x=nodeX,
        y=nodeY,
        mode="markers",
        hoverinfo="text",
        text=hoverText,
        marker=dict(
            size=12,
            color=nodeColors,
            line=dict(width=1, color="black")  # Black border
        )
    )

    # Build and show figure
    fig = go.Figure(
        data=[edgeTrace, nodeTrace],
        layout=go.Layout(
            title="Topologically Sorted Dependency Graph",
            titlefont_size=16,
            showlegend=False,
            hovermode="closest",
            width=1600,
            height=900,
            margin=dict(b=0, l=0, r=0, t=40),
            xaxis=dict(showgrid=False, zeroline=False),
            yaxis=dict(showgrid=False, zeroline=False),
        )
    )

    fig.show()



In [7]:
dpdFilePath: str = r"C:\Users\User\Documents\GitHub\autoformalization\src\dependency_graph\EverythingLF.dpd"

G = createGraphFromDpd(dpdFilePath)

plotGraphSpringLayout(G, title="Dependency Graph (Spring Layout)")

plotNodeDegreeHistogram(dpdFilePath)

plotTopologicallySortedGraph(dpdFilePath)


In [8]:
import re
import pandas as pd
import json

# Load JSON data from a file
with open(r"C:\Users\User\Documents\GitHub\autoformalization\src\dataset\processed_data\coq_proofs_dataset.json", "r") as file:
    json_data = json.load(file)

# Flatten JSON into a DataFrame
data = []
for entry in json_data:
    file_name = entry["fileName"]
    for item in entry["items"]:
        data.append({
            "fileName": file_name, 
            "type": item["type"], 
            "raw": item["raw"]
        })

df = pd.DataFrame(data)

def get_second_word(text):
    if text is None:
        return None
    # Split on whitespace; return second token if it exists
    tokens = text.split()
    if len(tokens) >= 2:
        return tokens[1].replace(':', '')  # Remove all instances of ':'
    else:
        return None


df["Label"] = df["raw"].apply(get_second_word)


In [9]:
def parse_attributes(attr_str):
    attr_str = attr_str.strip("[];")  # Remove brackets and semicolon
    attr_pairs = attr_str.split(", ")  # Split key-value pairs
    attr_dict = {}
    for pair in attr_pairs:
        if "=" in pair:
            key, value = pair.split("=")
            attr_dict[key.strip()] = value.strip().strip('"')  # Remove extra quotes
    return attr_dict

# Expand attributes for each node
expanded_data = []
for node, attributes in G.nodes(data=True):
    attr_dict = parse_attributes(attributes["attributes"])
    expanded_data.append({"Node": node, "Label": attributes["label"], **attr_dict})

# Create expanded DataFrame
df_dpd = pd.DataFrame(expanded_data)

# Function to extract filename from 'path' column
def extract_filename(path):
    return path.split(".")[0] + '.v' if path else None

# Create new column 'file' by extracting the filename
df_dpd["file"] = df_dpd["path"].apply(extract_filename)

# Display the DataFrame
df_dpd[['file','Label','Node']].head(10)
df_dpd


,Node,Label,body,kind,prop,path,file
0,644,All,no,cnst,no,EverythingLF.LF_DOT_Logic.LF.Logic,EverythingLF.v
1,643,All_In,no,cnst,yes,EverythingLF.LF_DOT_Logic.LF.Logic,EverythingLF.v
2,1072,Collatz_holds_for_12,yes,cnst,yes,EverythingLF.LF_DOT_IndProp.LF.IndProp,EverythingLF.v
3,1074,Collatz_holds_for_ind,yes,cnst,yes,EverythingLF.LF_DOT_IndProp.LF.IndProp,EverythingLF.v
4,1073,Collatz_holds_for_sind,yes,cnst,no,EverythingLF.LF_DOT_IndProp.LF.IndProp,EverythingLF.v
...,...,...,...,...,...,...,...
1762,940,MStarApp,NaN,construct,yes,EverythingLF.LF_DOT_IndProp.LF.IndProp,EverythingLF.v
1763,1749,BAnd,NaN,construct,no,EverythingLF.LF_DOT_Imp.LF.Imp.AExp,EverythingLF.v
1764,1320,BAnd,NaN,construct,no,EverythingLF.LF_DOT_Imp.LF.Imp,EverythingLF.v
1765,127,E_RepeatEnd,NaN,construct,yes,EverythingLF.LF_DOT_Auto.LF.Auto.Repeat,EverythingLF.v


In [10]:
merged_df = df.merge(df_dpd[['file', 'Label', 'Node']], 
                        left_on=['fileName', 'Label'], 
                        right_on=['file', 'Label'], 
                        how='left')

# Drop the redundant 'file' column (same as 'fileName')
merged_df.drop(columns=['file'], inplace=True)

# Fill missing 'Node' values with pd.NA (or use 'None' as a string if preferred)
merged_df['Node'] = merged_df['Node'].fillna(pd.NA)

# Identify unmatched records from df_dpd
unmatched_df_dpd = df_dpd[~df_dpd.set_index(['file', 'Label']).index.isin(df.set_index(['fileName', 'Label']).index)]


In [11]:
merged_df

,fileName,type,raw,Label,Node
0,AltAuto.v,global_directive,"Set Warnings ""-notation-overridden,-parsing,-d...",Warnings,<NA>
1,AltAuto.v,Import,From Coq Require Import Arith List.,Coq,<NA>
2,AltAuto.v,Import,From LF Require Import IndProp.,LF,<NA>
3,AltAuto.v,Fixpoint,Fixpoint re_opt_e {T:Type} (re: reg_exp T) : r...,re_opt_e,<NA>
4,AltAuto.v,Lemma,Lemma re_opt_e_match : forall T (re: reg_exp T...,re_opt_e_match,<NA>
...,...,...,...,...,...
2629,TacticsTest.v,misc,"idtac ""------------------- forall_exists_chal...","""-------------------",<NA>
2630,TacticsTest.v,misc,"idtac ""#> existsb_existsb'"".\nidtac ""Advanced""...","""#>",<NA>
2631,TacticsTest.v,misc,"Goal True.\nidtac "" "".",True.,<NA>
2632,TacticsTest.v,misc,"idtac "" "".","""",<NA>


In [12]:
unmatched_df_dpd

,Node,Label,body,kind,prop,path,file
0,644,All,no,cnst,no,EverythingLF.LF_DOT_Logic.LF.Logic,EverythingLF.v
1,643,All_In,no,cnst,yes,EverythingLF.LF_DOT_Logic.LF.Logic,EverythingLF.v
2,1072,Collatz_holds_for_12,yes,cnst,yes,EverythingLF.LF_DOT_IndProp.LF.IndProp,EverythingLF.v
3,1074,Collatz_holds_for_ind,yes,cnst,yes,EverythingLF.LF_DOT_IndProp.LF.IndProp,EverythingLF.v
4,1073,Collatz_holds_for_sind,yes,cnst,no,EverythingLF.LF_DOT_IndProp.LF.IndProp,EverythingLF.v
...,...,...,...,...,...,...,...
1762,940,MStarApp,NaN,construct,yes,EverythingLF.LF_DOT_IndProp.LF.IndProp,EverythingLF.v
1763,1749,BAnd,NaN,construct,no,EverythingLF.LF_DOT_Imp.LF.Imp.AExp,EverythingLF.v
1764,1320,BAnd,NaN,construct,no,EverythingLF.LF_DOT_Imp.LF.Imp,EverythingLF.v
1765,127,E_RepeatEnd,NaN,construct,yes,EverythingLF.LF_DOT_Auto.LF.Auto.Repeat,EverythingLF.v


In [13]:
# 1. Set label as second word in raw in coq_proofs_dataset.json    
# 2. parse node atr into a dict
# 3. extract filename from path
# 4. merge the two dataframes on label & file